In [18]:
import numpy as np
import pandas as pd

In [180]:
train_data=np.genfromtxt('training_ccpp_x_y_train.csv',delimiter=',')
test_data=np.genfromtxt('test_ccpp_x_test.csv',delimiter=',')

In [68]:
train_df=pd.read_csv('training_ccpp_x_y_train.csv',delimiter=',')
train_df.head()

,# T,V,AP,RH,EP
0,8.58,38.38,1021.03,84.37,482.26
1,21.79,58.20,1017.21,66.74,446.94
2,16.64,48.92,1011.55,78.76,452.56
3,31.38,71.32,1009.17,60.42,433.44
4,9.20,40.03,1017.05,92.46,480.38


In [70]:
tr_y=train_df.values[:,4]
tr_y

array([482.26, 446.94, 452.56, ..., 437.65, 459.97, 444.42])

In [79]:
del train_df[' EP']

In [80]:
train_data.shape

(7176, 5)

In [81]:
from sklearn import preprocessing

In [82]:
scaler=preprocessing.StandardScaler()

In [83]:
scaler.fit(train_df)

StandardScaler()

In [84]:
train_scale=scaler.transform(train_df)

In [85]:
test_scale=scaler.transform(test_data)

In [86]:
name=train_df.columns

In [87]:
index=name
train_df=pd.DataFrame(train_scale,columns=index)

In [88]:
index=[' EP']
df_y=pd.DataFrame(tr_y,columns=index)

In [90]:
df=train_df.join(df_y)

In [94]:
train_data=df.values

In [95]:
train_data

array([[-1.47827466e+00, -1.24764165e+00,  1.30221067e+00,
         7.58624590e-01,  4.82260000e+02],
       [ 2.89012041e-01,  3.06797549e-01,  6.61749044e-01,
        -4.46921842e-01,  4.46940000e+02],
       [-3.99975582e-01, -4.21012529e-01, -2.87207194e-01,
         3.75010552e-01,  4.52560000e+02],
       ...,
       [ 1.36062192e+00,  1.18048335e+00, -6.54382840e-01,
        -5.83682640e-01,  4.37650000e+02],
       [-4.36097263e-01,  9.29089763e-04,  7.84140927e-01,
        -6.59584883e-01,  4.59970000e+02],
       [ 1.40209496e+00,  6.07960340e-01, -4.31394890e-01,
        -1.73110573e+00,  4.44420000e+02]])

In [126]:
def cost(points,m):
    total_cost=0
    N=points.shape[0]
    c=points.shape[1]
    for i in range(N):
        x=points[i,0:c-1]
        x=np.append(x,1)
        y=points[i,c-1]
        total_cost+=(y-(m*x).sum())**2
    total_cost=(1/N)*total_cost
    return total_cost

In [ ]:
def step_gradient(points,learn_rate,m):
    c=points.shape[1]
    m_slope=np.zeros(c)
    M=points.shape[0]
    new_m=0
    for i in range(M):
        x=points[i,0:c-1]
        x=np.append(x,1)
        y=points[i,c-1]
        for j in range(c):
            m_slope[j]+=(-2/M)*(y-(m*x).sum())*x[j]
        new_m=m-(learn_rate*m_slope)
    return new_m       

In [134]:
def gd(points,num_iter,learn_rate):
    c=points.shape[1]
    m=np.zeros(c)
    for i in range(num_iter):
        m=step_gradient(points,learn_rate,m)
        print(i,' ',"Cost:" ,cost(points,m))
    return m

In [162]:
def run():
    num_iter=100
    learn_rate=0.0001
    m=gd(train_data,num_iter,learn_rate)
    return m[0:train_data.shape[1]-1], m[train_data.shape[1]-1] #no of rows in traing data

In [163]:
m,c=run()

0   Cost: 27422.391537749892
1   Cost: 3661.741184229716
2   Cost: 509.4162439398636
3   Cost: 90.679865576776
4   Cost: 34.36609667266653
5   Cost: 26.298115850695876
6   Cost: 24.711342779573602
7   Cost: 24.063822257310964
8   Cost: 23.60319883249452
9   Cost: 23.22127102304074
10   Cost: 22.89527361542579
11   Cost: 22.615669698410695
12   Cost: 22.375561062899454
13   Cost: 22.169300129563187
14   Cost: 21.99208784322746
15   Cost: 21.83982674403131
16   Cost: 21.709004737632025
17   Cost: 21.59660814698133
18   Cost: 21.500048515549384
19   Cost: 21.4171016519018
20   Cost: 21.34585580381125
21   Cost: 21.284667512958013
22   Cost: 21.232123831265003
23   Cost: 21.187009933797146
24   Cost: 21.148281318950268
25   Cost: 21.115039930080368
26   Cost: 21.086513635387
27   Cost: 21.062038588616364
28   Cost: 21.04104406313109
29   Cost: 21.023039410871657
30   Cost: 21.007602847529906
31   Cost: 20.994371807677126
32   Cost: 20.983034649788788
33   Cost: 20.973323522101623
34   Cost

In [165]:
print("M :",m)
print("C :",c)

M : [-14.90685845  -2.84188676   0.36061708  -2.38395224]
C : 454.4575257440056


In [176]:
def prediction(points,m,c):
    r=points.shape[0]
    pred=np.zeros(r)
    for i in range(r):
        x=points[i,:]
        pred[i]=(m*x).sum() +c
    return pred
    

In [177]:
predict=prediction(test_data,m,c)

In [178]:
predict

array([307.15537856, 336.34213685,  14.5672917 , ...,  48.48145086,
       192.61804385, 123.47917577])

In [179]:
np.savetxt('predction_cycle.csv',predict,delimiter=',',fmt="%1.5f")

In [181]:
train_data

array([[   8.58,   38.38, 1021.03,   84.37,  482.26],
       [  21.79,   58.2 , 1017.21,   66.74,  446.94],
       [  16.64,   48.92, 1011.55,   78.76,  452.56],
       ...,
       [  29.8 ,   69.34, 1009.36,   64.74,  437.65],
       [  16.37,   54.3 , 1017.94,   63.63,  459.97],
       [  30.11,   62.04, 1010.69,   47.96,  444.42]])

In [182]:
test_data

array([[  11.95,   42.03, 1017.58,   90.89],
       [  12.07,   38.25, 1012.67,   81.66],
       [  26.91,   74.99, 1005.64,   78.98],
       ...,
       [  24.32,   66.25, 1009.09,   91.89],
       [  23.49,   42.8 , 1013.96,   65.31],
       [  21.76,   60.27, 1018.96,   85.06]])

In [194]:
from sklearn.ensemble import GradientBoostingRegressor

In [184]:
x=train_data[:,:-1]
y=train_data[:,-1]

In [185]:
x.shape

(7176, 4)

In [186]:
y.shape

(7176,)

In [187]:
train_data.shape

(7176, 5)

In [188]:
reg=GradientBoostingRegressor()

In [189]:
reg.fit(x,y)

GradientBoostingRegressor()

In [195]:
y_predict=reg.predict(x)

In [196]:
np.savetxt('prediction_final_cycle.csv',y_predict,delimiter=',',fmt="%1.5f")

In [197]:
y_predict

array([480.91549752, 449.27889301, 457.49658362, ..., 433.68679481,
       460.0522543 , 439.74172019])